# Project 1: Email Classifier
- Chloe Wallach
- Roberto Garcia
- Grant Fass

## Imports

In [2]:
import pandas as pd
import numpy as np
import os
import json
import glob


## Load the Data

In [5]:
base_directory = os.path.abspath("./email_json_dataset1")
dir_exists = os.path.exists(base_directory)
file_names = []
if dir_exists:
    file_names = glob.glob(base_directory + "/**")
    print("Num JSON Files: %d" % (len(file_names)))


Num JSON Files: 63542


In [7]:
emails = []
for file in file_names:
    f = open(file)
    dictionary = json.load(f)
    emails.append(dictionary)

print("Type: %s, Len %d" % (type(emails), len(emails)))

Type: <class 'list'>, Len 63542


In [8]:
df = pd.DataFrame.from_records(emails)
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63542 entries, 0 to 63541
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   label         63542 non-null  object
 1   to_address    63141 non-null  object
 2   from_address  63542 non-null  object
 3   subject       63410 non-null  object
 4   body          63542 non-null  object
dtypes: object(5)
memory usage: 2.4+ MB


In [9]:
df.head(5)

,label,to_address,from_address,subject,body
0,spam,the00@speedy.uwaterloo.ca,"""Tomas Jacobs"" <RickyAmes@aol.com>","Generic Cialis, branded quality@",\n\n\n\n\n\n\nDo you feel the pressure to perf...
1,ham,debian-mirrors@lists.debian.org,Yan Morin <yan.morin@savoirfairelinux.com>,Typo in /debian/README,"Hi, i've just updated from the gulus and I che..."
2,spam,<the00@plg.uwaterloo.ca>,"""Sheila Crenshaw"" <7stocknews@tractionmarketin...",authentic viagra,authentic viagra\n\nMega authenticV I A G R A...
3,spam,opt4@speedy.uwaterloo.ca,"""Stormy Dempsey"" <vqucsmdfgvsg@ruraltek.com>",Nice talking with ya,"\nHey Billy, \n\nit was really fun going out t..."
4,spam,ktwarwic@speedy.uwaterloo.ca,"""Christi T. Jernigan"" <dcube@totalink.net>",or trembling; stomach cramps; trouble in sleep...,"\n\n\n\n\n\n\nsystem"" of the home. It will ha..."
